# Uji Coba Code dari MediaPipe dan FaceNet

#### Import Library

In [3]:
import cv2
import mediapipe as mp
import os
import dlib


#### Inisiasi Wajah dan Drawing dari Media Pipe

In [9]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(255, 0, 0))  # Warna biru (BGR)


#### Convert Gambar menjadi Gambar yang ada Face Landmark MediaPipe

In [11]:
def process_image(image_path, output_folder):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Gambar landmark pada wajah (opsional)
            mp_drawing.draw_landmarks(img, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION, drawing_spec, drawing_spec)

            # Simpan landmark ke dalam file teks
            filename = os.path.basename(image_path).split('.')[0] + '.txt'
            output_path = os.path.join(output_folder, filename)
            with open(output_path, 'w') as f:
                for landmark in face_landmarks.landmark:
                    f.write(f'{landmark.x} {landmark.y} {landmark.z}\n')

            # Simpan gambar yang telah ditandai (opsional)
            output_image_path = os.path.join(output_folder, filename[:-4] + '.jpg')
            cv2.imwrite(output_image_path, img)
    else:
        print(f"Tidak ada wajah terdeteksi di {image_path}")

#### Eksport Hasil Labelling Dengan MediaPipe

In [12]:
dataset_folder = r"\program-TA-master\Dataset\Test"
output_folder = r"\program-TA-master\Dataset\Hasil\TestMediaPipe"

# Pastikan folder output существует
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(dataset_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(dataset_folder, filename)
        process_image(image_path, output_folder)

In [6]:
# Path ke model dlib (ganti dengan path Anda)
predictor_path = "\program-TA-master\Model\shape_predictor_68_face_landmarks.dat"  # Sesuaikan dengan path Anda
detector = dlib.get_frontal_face_detector() #inisialisasi detector wajah
predictor = dlib.shape_predictor(predictor_path) #inisialisasi predictor landmark

#### DLIB Dataset

In [7]:
def process_image2(image_path, output_folder):
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error reading image: {image_path}")
            return

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        dets = detector(gray, 1)

        for det in dets:
            x1, y1, x2, y2 = det.left(), det.top(), det.right(), det.bottom()

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Gambar bounding box

            shape = predictor(gray, det)
            for i in range(68):
                x = shape.part(i).x
                y = shape.part(i).y
                cv2.circle(img, (x, y), 2, (0, 255, 255), -1)  # Gambar landmark (kuning)

            output_filename = os.path.basename(image_path)
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, img)

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

In [8]:
dataset_folder = r"\program-TA-master\Dataset\Test"
output_folder = r"\program-TA-master\Dataset\Hasil\TestDILB"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(dataset_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(dataset_folder, filename)
        process_image2(image_path, output_folder)